env: cell_rank

# import

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
# import torch
# from torch_geometric.data import Data
# from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
# from scperturb import *
import anndata as ad
import scvelo as scv

Global seed set to 0


# initial

In [2]:
dataset_pert_dict = {
    'CAR_T': ['PDCD1'], # direct_transfer只有前两个数据有
    'blood': ['GATA1', 'SPI1'],
    # 'OSKM': ['SOX2',
    #      'POU5F1',
    #      'KLF4',
    #      'MYC'],
    # 'ADM': ['PTF1A']
}
dataset_celltype_dict = {
    'CAR_T': 'Tex', 
    'blood': 'LMPP',
    'OSKM': 'Fibroblast-like',
    'ADM': 'Acinar'
}

dataset_dire_dict = {
    'CAR_T': 'down', 
    'blood': 'down',
    'OSKM': 'up',
    'ADM': 'down'
}


datasets = list(dataset_pert_dict.keys())

In [3]:
import json
with open('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/real_case/utils_data/direct_change_dict.json', 'r') as f:
    direct_change_dict = json.load(f)

# direct_tranfer跑所有datasets

In [4]:
for dataset in datasets[:]:
    # datasets = 'CAR_T'
    print('='*20, dataset, '='*20)
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/real_case/data'
    adata_rna = sc.read(os.path.join(save_dir, dataset, 'adata_ctrl_v2.h5ad'))
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    adata = adata_rna.copy()

    for pert in tqdm(dataset_pert_dict[dataset]):
        if pert not in direct_change_dict:
            print(f'{pert} not in direct_transfer_dict')
        print('*'*20, pert)

        # - direct transfer
        adata_ctrl = adata_rna.copy()

        gene_list = direct_change_dict[pert]['gene_list']
        direct_change = direct_change_dict[pert]['direct_change']

        exp_change = np.zeros(adata_ctrl.shape[1])

        print(f'{pert} common var is {len(np.intersect1d(gene_list, adata_ctrl.var_names))} / {len(adata_ctrl.var_names)}')
        for i in range(len(exp_change)):
            if adata_ctrl.var_names[i] in gene_list:
                exp_change[i] = direct_change[gene_list.index(adata_ctrl.var_names[i])]

        adata_pert = adata_ctrl.copy()
        adata_pert.X += exp_change
        adata_pert.X[adata_pert.X < 0] = 0
        adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

        # - save
        tmp_dir = f'/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/real_case/result/{dataset}'
        save_prefix = f'direct_transfer/{pert}' # use result of K562 to do the direct transfer
        os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)
        adata_pert.write(os.path.join(tmp_dir, save_prefix, 'adata_pert.h5ad'))

    # break

==================== CAR_T ====================


  0%|          | 0/1 [00:00<?, ?it/s]

******************** PDCD1
PDCD1 common var is 2524 / 2647


100%|██████████| 1/1 [00:03<00:00,  3.44s/it]


==================== blood ====================


  0%|          | 0/2 [00:00<?, ?it/s]

******************** GATA1
GATA1 common var is 571 / 1206


 50%|█████     | 1/2 [00:49<00:49, 49.81s/it]

******************** SPI1
SPI1 common var is 1096 / 1206


100%|██████████| 2/2 [02:00<00:00, 60.03s/it]
